In [6]:
from bs4 import BeautifulSoup
from requests import get
import random
import string
import sqlite3
import json
import logging

# Input
already_file = 'already_links_in/telegraph.txt'
current_reviews = set(line.strip() for line in open(already_file))

# Output
log_path = '../Logs/test.log'
db_path = '../Data/test.db'
db_name = 'test'
conn = sqlite3.connect(db_path)
c = conn.cursor()
#
logger = logging.getLogger()
fhandler = logging.FileHandler(filename=log_path, mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

def generateNode(length):
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    node = "_:znode" + result_str
    return node

already_count = 0
no_annotation = 0
review_count = 0

review_link ='https://www.telegraph.co.uk/films/0/avengers-endgame-review-spoilers-end-credits-ending-explained/'
if (review_link in current_reviews):
    already_count += 1
    pass
else:
    review_soup = BeautifulSoup(get(review_link).text, 'lxml')
    has_annotation = review_soup.find('main', itemtype='https://schema.org/Review')
    if has_annotation is None:
        no_annotation += 1
        pass
    else:
        review = has_annotation.find('article', itemprop='reviewBody')
        body = review.find_all('p')
        reviewBody = ""
        for item in body:
            reviewBody += reviewBody + item.text
        reviewRating = "already included"
        ratingValue = has_annotation.find('span', itemprop='reviewRating').find('span', class_='entity-property-numeric-score').text
        worstRating = "1"
        bestRating = "5"
        node = generateNode(31)
        try:
            c.execute(f"INSERT OR IGNORE INTO {db_name} (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);",(node,review_link,reviewBody,str(reviewRating),ratingValue,bestRating,worstRating))
            conn.commit()
        except Exception as e:
            logging.debug("Error at " + str(review_link))
            logging.debug("Länge des Strings " + str(len(reviewBody)))
            raise(e)
    review_count += 1

logging.debug("Reviews extracted: " + str(review_count))
logging.debug("Reviews without Review Annotation: " + str(no_annotation))
logging.debug("Already included: " + str(already_count))

OverflowError: string longer than INT_MAX bytes

In [109]:
fail_set = set()
fail_set.add('https://www.empireonline.com/movies/reviews/love-wedding-repeat/')
fail_set.add('https://www.empireonline.com/movies/reviews/downhill/')
fail_set.add('https://www.empireonline.com/movies/reviews/black-christmas-2019/')
fail_set.add('https://www.empireonline.com/movies/reviews/i-lost-my-body/')

In [124]:
while len(fail_lst)>0:
    for i in fail_lst:
        review_link = i
        response = get(review_link)
        if response.status_code == 404:
            pass
        else:
            try:
                review_soup = BeautifulSoup(response.text, 'lxml')
                data = json.loads("".join(review_soup.find("script", {"type": "application/ld+json"}).contents),strict=False)
                if not data['@type'] == "review":
                    no_annotation += 1
                    pass
                else:
                    reviewBody = data['description'].replace('\n','')
                    worstRating = data['reviewRating']['worstRating']
                    bestRating = data['reviewRating']['bestRating']
                    ratingValue = data['reviewRating']['ratingValue']
                    reviewRating = "already included"
                    node = generateNode(31)
                    c.execute(f"INSERT OR IGNORE INTO {db_name} (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);", (node, review_link, reviewBody, str(reviewRating), ratingValue, bestRating, worstRating))
                    conn.commit()
                    review_count += 1
                    fail_lst.remove(review_link)
            except Exception as e:
                logging.debug("Error: " + str(review_link))
                logging.debug(str(review_soup))
                raise(e)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.empireonline.com:443
DEBUG:urllib3.connectionpool:https://www.empireonline.com:443 "GET /movies/reviews/love-wedding-repeat/ HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.empireonline.com:443
DEBUG:urllib3.connectionpool:https://www.empireonline.com:443 "GET /movies/reviews/i-lost-my-body/ HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.empireonline.com:443
DEBUG:urllib3.connectionpool:https://www.empireonline.com:443 "GET /movies/reviews/downhill/ HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.empireonline.com:443
DEBUG:urllib3.connectionpool:https://www.empireonline.com:443 "GET /movies/reviews/black-christmas-2019/ HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.empireonline.com:443
DEBUG:urllib3.connectionpool:https://www.empireonline.com:443 "GET /movies/rev

node:_:znode5O61iJ0AW6bVif0SlhJtrHV8tabWgJs
url: https://www.empireonline.com/movies/reviews/love-wedding-repeat/
reviewBody: Itâs Sliding Doors with place settings, but Love Wedding Repeat canât make its time-loop conceit work (stick with About Time). Bouquets to the cast and production values; a quickie divorce from everything else.
worstRating: 1
bestRating: 5
ratingValue: 2
node:_:znodeRgEtW3iSxnRjgs7v3MZDInsYBTEHd8m
url: https://www.empireonline.com/movies/reviews/i-lost-my-body/
reviewBody: If itâs slightly hampered by a generic love story and pie-in-the-sky teenage pontificating, I Lost My Body should still rank among the yearâs most original and peculiar films. Hands down.
worstRating: 1
bestRating: 5
ratingValue: 3
node:_:znodeoZAYF375M574Iscawz1Z4gWZb8ZBaCd
url: https://www.empireonline.com/movies/reviews/downhill/
reviewBody: Like a lot of remakes, Downhill doesnât quite make the argument for its existence. But career-topping performances from Ferrell and Louis-Dre

In [125]:
for i in fail_lst:
    review_link = i
    print(review_link)

In [122]:
fail_lst.append('https://www.empireonline.com/movies/reviews/love-wedding-repeat/')
fail_lst.append('https://www.empireonline.com/movies/reviews/black-christmas-2019/')
fail_lst.append('https://www.empireonline.com/movies/reviews/i-lost-my-body/')
fail_lst.append('https://www.empireonline.com/movies/reviews/the-day-shall-come/')
fail_lst.append('https://www.empireonline.com/movies/reviews/downhill/')

In [119]:
print(len(fail_lst))



5
